In [14]:
import os
from pathlib import Path
import anndata as ad
from scipy import sparse
import scanpy as sc

In [3]:
os.chdir(Path().resolve())
os.getcwd()

'/home/wu/datb1/AutoExtractSingleCell/03.skin_phase2/sample2/raw_data'

In [18]:
if 'adata_all' in locals():
    del adata_all

sample_set = os.listdir('.')
for sample in sample_set:
    if sample.endswith('csv.gz'):
        adata = sc.read_text(sample, delimiter=',').T
        adata.X = sparse.csr_matrix(adata.X)
        print(f"Processing sample {sample}")
        print(adata.shape)
        # drop var col
        # adata.var = pd.DataFrame(index=adata.var_names)
        adata.obs['Batch'] = sample.replace('.csv.gz', '')
        adata.var_names_make_unique()
        adata.obs_names_make_unique()
        adata.obs_names = adata.obs_names + '_' + adata.obs['Batch']
        # if adata_all is already defined, concat the new adata to it
        if 'adata_all' in locals():
            adata_all = ad.concat([adata_all, adata], uns_merge='unique')
        else:
            adata_all = adata.copy()
            adata_all.uns['batch'] = []
            adata_all.uns['batch_raw'] = {}

        del adata
        
        adata_all.uns['batch'].append(sample)
        adata_all.uns['batch_raw'][sample] = len(adata_all.obs_names)
        # adata_all.write('sample2_raw.h5ad')

Processing sample GSM5277188_PN-9709PC.csv.gz
(927, 33538)
Processing sample GSM5277173_NS-AR005.csv.gz
(1064, 33538)
Processing sample GSM5277200_PP-8940.csv.gz
(3929, 33538)
Processing sample GSM5277171_NS-AR003.csv.gz
(2345, 33538)
Processing sample GSM5277174_NS-AR006.csv.gz
(2438, 33538)
Processing sample GSM5277190_PP-30696V3.csv.gz
(2101, 33538)
Processing sample GSM5277186_PN-8940.csv.gz
(5577, 33538)
Processing sample GSM5277185_PN-8659ED.csv.gz
(3273, 33538)
Processing sample GSM5277193_PP-31499.csv.gz
(2843, 33538)
Processing sample GSM5277187_PN-929.csv.gz
(5216, 33538)
Processing sample GSM5277191_PP-31170.csv.gz
(3746, 33538)
Processing sample GSM5277181_PN-31277.csv.gz
(3522, 33538)
Processing sample GSM5277184_PN-7802ED.csv.gz
(5010, 33538)
Processing sample GSM5277177_NS-AR009.csv.gz
(2295, 33538)
Processing sample GSM5277202_PP-9709PC.csv.gz
(2994, 33538)
Processing sample GSM5277192_PP-31277.csv.gz
(3044, 33538)
Processing sample GSM5277199_PP-8659ED.csv.gz
(3177, 33

In [23]:
adata_all.obs['Disease'] = ''
adata_all.obs.loc[adata_all.obs['Batch'].str.contains('PP'), 'Disease'] = 'Psor'
adata_all.obs.loc[adata_all.obs['Batch'].str.contains('PN'), 'Disease'] = 'Psor Normal'
adata_all.obs.loc[adata_all.obs['Batch'].str.contains('NS'), 'Disease'] = 'Healthy'

In [24]:
adata_all.obs['Disease'].value_counts()

Disease
Psor           47036
Psor Normal    35518
Healthy        13534
Name: count, dtype: int64

In [25]:
adata_all.write('sample2_raw.h5ad')